In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
def read_file(x):
    #read in the csv file with low memory
    y = pd.read_csv(x, low_memory=False)
    return y

In [13]:
def process_data(df):
    df['patientunitstayid'] = df['patientunitstayid'].astype(int)
    df = df[df['nursingchartvalue'] != 'Unable to score due to medication']
    df['nursingchartvalue'] = pd.to_numeric(df['nursingchartvalue'], errors='coerce').astype(float)
    df = df[~df['cellattributevalue'].isin(['feet', 'hands'])]
    # copy all complete numbers and non-nan values at the beginnning of the dataset
    d = df[['admissionheight', 'admissionweight', 'age', 'gender', 'patientunitstayid', 'unitvisitnumber']].copy()
    #encode genders to get unique instances
    one_hot = pd.get_dummies(d['gender'])
    #use one hot encoding to generate dummies and catagorize the dataset
    d = pd.concat([d, one_hot], axis=1)
    #drop gender column
    d.drop('gender', axis=1, inplace=True)
    # replace the nonfloats/itns with ints 
    d.replace({'> 89': 100}, inplace = True)
    d = d.drop_duplicates(subset=['patientunitstayid'], keep='first')
    d = d.set_index('patientunitstayid')
    d['cellattributevalue'] = 0
    
    #copy another instence on the next part of the dataset
    d2 = df[['cellattributevalue', 'celllabel', 'offset', 'patientunitstayid']].copy()
    d2.dropna(inplace=True)
    #replace all nonfloat values with integers based off of numerics
    d2.replace({'< 2 seconds': 0}, inplace = True)
    d2.replace({'> 2 seconds': 1}, inplace = True)
    d2.replace({'normal': 2}, inplace = True)
    #index the patient ids with the cell atttributes to generate values in the finalized dataset
    for idx, row in d2.iterrows():
        id = row['patientunitstayid']
        if id in d.index and pd.isna(d.at[id, 'cellattributevalue']):
            d.at[id, 'cellattributevalue'] = row['cellattributevalue']
    #generate new columns         
    columns = ['pH', 'glucose']
    d[columns] = None
    
    d3 = df[['labname','labresult','offset','patientunitstayid']].copy()
    d3.dropna(inplace=True)
    #index the patient ids with the cell atttributes to generate values in the finalized dataset
    for i, row in d3.iterrows():
        id = row['patientunitstayid']
        labname = row['labname']
        labresult = row['labresult']
        if id in d.index:
            if pd.isna(d.loc[id, labname]):
                d.loc[id, labname] = labresult
            else:
                d.loc[id, labname] = min(d.loc[id, labname], labresult)
                
    #generate new columns             
    columns = ['GCS Total', 'Heart Rate', 'Non-Invasive BP Diastolic', 'Non-Invasive BP Mean', 'Non-Invasive BP Systolic', 'Invasive BP Diastolic', 'Invasive BP Mean', 'Invasive BP Systolic', 'Respiratory Rate', 'O2 Saturation']
    d[columns] = None
    #copy another instence on the last part of the dataset
    d4 = df[['nursingchartcelltypevalname','nursingchartvalue','offset','patientunitstayid']].copy()
    
    d4.dropna(inplace=True)
    #index the patient ids with the cell atttributes to generate values in the finalized dataset
    for i, row in d4.iterrows():
        id = row['patientunitstayid']
        if id in d.index:
            current_value = d.loc[id, row['nursingchartcelltypevalname']]
            new_value = row['nursingchartvalue']
            if current_value is None or current_value > new_value:
                d.loc[id, row['nursingchartcelltypevalname']] = new_value
    #convert all types to floats     
    d = d.astype(float)
    #get all nans to be imputed with the mean datasets for the rows
    d = d.fillna(d.mean())
    finalized = d
    return finalized

In [4]:
#establish file paths
x_train_file_path = 'train_x.csv/train_x.csv'
x_test_file_path = 'test_x.csv/test_x.csv'
y_train_file_path = 'train_y.csv'     

In [5]:
#read in the respective file paths to data frames
train_y = read_file(y_train_file_path)
train_x = read_file(x_train_file_path)
test_x = read_file(x_test_file_path)    

In [6]:
#check the dataset to make sure it remains a dataframe
type(train_x) 

pandas.core.frame.DataFrame

In [7]:
#process the data accordingly 
train_x = process_data(train_x)
test_x = process_data(test_x)      

C:\Users\alexd\AppData\Local\Temp\ipykernel_11932\1601349088.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nursingchartvalue'] = pd.to_numeric(df['nursingchartvalue'], errors='coerce').astype(float)
C:\Users\alexd\AppData\Local\Temp\ipykernel_11932\1601349088.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nursingchartvalue'] = pd.to_numeric(df['nursingchartvalue'], errors='coerce').astype(float)


In [8]:
#show the dataset to make sure it all matches the correct format
train_x

,admissionheight,admissionweight,age,unitvisitnumber,Female,Male,cellattributevalue,pH,glucose,GCS Total,Heart Rate,Non-Invasive BP Diastolic,Non-Invasive BP Mean,Non-Invasive BP Systolic,Invasive BP Diastolic,Invasive BP Mean,Invasive BP Systolic,Respiratory Rate,O2 Saturation
patientunitstayid,,,,,,,,,,,,,,,,,,,
141764,157.5,82.688572,87.0,2.0,1.0,0.0,0.0,7.301414,109.246531,12.549434,98.0,78.0,103.000000,153.0,45.550162,61.965035,93.23301,12.198135,88.000657
141765,157.5,46.500000,87.0,1.0,1.0,0.0,0.0,7.301414,61.000000,15.000000,74.0,59.0,75.000000,120.0,45.550162,61.965035,93.23301,14.000000,96.000000
143870,167.0,77.500000,76.0,1.0,0.0,1.0,0.0,7.390000,123.000000,15.000000,40.0,34.0,63.091616,93.0,30.000000,61.965035,100.00000,12.000000,94.000000
144815,172.7,60.300000,34.0,1.0,1.0,0.0,0.0,7.400000,82.000000,12.000000,68.0,46.0,61.000000,82.0,45.550162,61.965035,93.23301,7.000000,94.000000
145427,177.8,91.700000,61.0,1.0,0.0,1.0,0.0,7.301414,118.000000,15.000000,49.0,47.0,63.091616,79.0,45.550162,61.965035,93.23301,9.000000,90.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3351297,159.0,77.300000,52.0,1.0,0.0,1.0,0.0,7.413000,96.000000,12.549434,42.0,58.0,78.000000,109.0,45.550162,61.965035,93.23301,12.000000,78.000000
3351763,165.1,134.500000,62.0,1.0,1.0,0.0,0.0,7.375000,87.000000,8.000000,91.0,42.0,61.000000,92.0,40.000000,55.000000,85.00000,18.000000,91.000000
3352231,177.8,127.000000,41.0,1.0,0.0,1.0,0.0,7.393000,85.000000,12.549434,51.0,60.0,77.000000,105.0,45.550162,61.965035,93.23301,13.000000,93.000000


In [9]:
# Define the GradientBoostingRegressor with the specified hyperparameters
gb = GradientBoostingRegressor(n_estimators=400,
                                learning_rate=0.01,
                                max_depth=5,
                                min_samples_leaf=4,
                                min_samples_split=2,
                                subsample=0.25,
                                random_state=42)

# Fit the GradientBoostingRegressor to the training data
gb.fit(train_x, train_y['hospitaldischargestatus'])

# Use the trained model to predict the target variable for the test data
y_pred = gb.predict(test_x)

# Create a new DataFrame containing the patientunitstayid column from the test data
done = test_x.filter(['patientunitstayid'])

# Add the predicted hospitaldischargestatus values to the done DataFrame
done['hospitaldischargestatus'] = y_pred

# Save the done DataFrame to a CSV file
done.to_csv('result.csv')

In [10]:
# print the done dataframe for results
done

,hospitaldischargestatus
patientunitstayid,
151179,0.211204
151900,0.017816
152954,0.005031
158056,0.030466
159411,-0.013227
...,...
3343295,0.008176
3344944,0.119303
3347502,0.007765


Below is code imported from chatgpt to optimize hyperparams for GradientBoostingRegressor as this is the only ML model this project is using for simplicity purposes. 

In [11]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston

def optimize_hyperparams():
    # Load the Boston housing dataset
    X = train_x 
    y = train_y['hospitaldischargestatus']

    # Define the hyperparameters to tune
    hyperparams = {
        'n_estimators': [400,500,600,700],
        'learning_rate': [0.1, 0.01, 0.001],
        'max_depth': [3, 4, 5],
        'min_samples_split': [2],
        'min_samples_leaf': [4],
        'subsample': [0.25,0.5,0.75],
        'random_state': [42]
    }

    # Define the model
    model = GradientBoostingRegressor()

    # Perform grid search to find the best hyperparameters
    grid_search = GridSearchCV(
        model,
        hyperparams,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        cv=5
    )

    grid_search.fit(X, y)

    # Print the best hyperparameters and score
    print('Best hyperparameters:', grid_search.best_params_)
    print('Best score:', -1 * grid_search.best_score_)

In [15]:
# this is the ouput of the hyperparameters according to the above funciton
optimize_hyperparams() 

Best hyperparameters: {'learning_rate': 0.01, 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 400, 'random_state': 42, 'subsample': 0.25}
Best score: 0.05491771280225112
